In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from main.tomography import Hamiltonian_Learning, Pauli_Transfer_Matrix 

Let us begin configuring the __backend__, which may be an ideal simulator, a noisy simulator, or a real backend.

In [ ]:
# # ## For ideal simulations
# from qiskit_aer import AerSimulator
# backend = AerSimulator()

In [ ]:
# # For noisy simulations
# from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
# backend = FakeBrisbane()
# backend.num_qubits 

In [ ]:
## For experiment
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel='ibm_cloud', 
                                token='')
# backend = service.least_busy(operational=True, simulator=False)
backend = service.backend('ibm_brisbane')
backend

In [ ]:
# qubits_layaout = [0,1,2,3,4,5,6] #7qubits

# qubits_layaout = [19,18,14,
#             0,1,2,3,4,5,6,7,8,9,10,11,12,17,
#             30,29,28,27,26,25,24,23,22,21,20,33,
#             39,40,41,42,43,44,45,46,47,48,49,55,
#             ] #41 qubits

qubits_layaout = [13,12,11,10,9,8,7,6,5,4,3,2,1,0,14,
                    18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,36,
                    51,50,49,48,47,46,45,44,43,42,41,40,39,38,37,52,
                    56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,74,
                    89,88,87,86,85,84,83,82,81,80,79,78,77,76,75,90,
                    94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,112,
                    126,125,124,123,122,121,120,119,118,117,116,115,114,113] #109

len(qubits_layaout)

The experiment in real hardware will be executed with __dynamical decoupling__ and __randomized compiling__ to mitigate the experimental errors.

In [ ]:
# We are going to execute the circuit with a Sampler with dynamical decoupling and randomized compiling
sampler = Sampler( mode=backend ) 
# sampler.options.dynamical_decoupling.enable = True
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = 1
sampler.options.twirling.shots_per_randomization = 100
# sampler.options.twirling.strategy = 'active'


Setting the pass manager.

In [ ]:
# We create two pass_maganers, one for optimizing the theoretical circuit and another for embedding into the hardware
pm0 = generate_preset_pass_manager( optimization_level=3 )
pm  = generate_preset_pass_manager(backend=backend,
                                    optimization_level=2,
                                    initial_layout=qubits_layaout )

We are going to simulate the evolution of the $N$-qubit Hamiltonian

$$ H = \sum_{j=0}^{N} h_j Z_j + \sum_{j=1}^{N-1} J_j X_j X_{j+1} .$$



In [ ]:
N = 10 # trotter steps 
num_qubits = len(qubits_layaout) 
hs = [ 1/8 for j in range(num_qubits) ] # parameters h_j
Js = [ 1/16 for j in range(num_qubits-1) ] # Parameters J_j

We will employ the Trotter formula for $t=1$. 

$$ e^{-itH} \approx e^{-i\Delta tH}\times e^{-i\Delta tH} \times \cdots\times e^{-i\Delta tH}.$$ 

Here we create the circuit for a single Trotter step.

In [ ]:
trotter_step = QuantumCircuit( num_qubits )

for j in range(0, num_qubits):
    trotter_step.rz( 2*hs[j]/N, j ) 
for j in range(0,num_qubits-1,2):
    trotter_step.rxx( 2*Js[j]/N, j, j+1 ) 
for j in range(1,num_qubits-1,2):
    trotter_step.rxx( 2*Js[j]/N, j, j+1 ) 

trotter_step.draw('mpl', fold=-1) 

Initiating the class for Hamiltonian Learning (HL).

In [ ]:
hamil_learn = Hamiltonian_Learning( num_qubits )

This cell create and transpile the circuits for the HL.

In [ ]:
qc_tomo_group_prepm = hamil_learn.circuits_tomo( trotter_step, N )
qc_tomo_group_prepm = pm0.run( qc_tomo_group_prepm )

qc_tomo_group = pm.run( qc_tomo_group_prepm ) 

In [ ]:
# qc_tomo_group_prepm[0].draw('mpl', fold=-1, idle_wires=False )  

In [ ]:
print( qc_tomo_group[0].depth() )
# qc_tomo_group[0].draw('mpl',fold=-1, idle_wires=False)  

This cell expects to execute the circuits.

In [ ]:
for j in range(2):
    
    # We are going to execute the circuit with a Sampler with dynamical decoupling and randomized compiling
    sampler = Sampler( mode=backend ) 
    # sampler.options.dynamical_decoupling.enable = True
    sampler.options.twirling.enable_gates = True
    sampler.options.twirling.num_randomizations = 1
    sampler.options.twirling.shots_per_randomization = 100
    # sampler.options.twirling.strategy = 'active'

    job = sampler.run( qc_tomo_group ) 
    print(job.job_id()) 

In [ ]:
from time import sleep
while not ( job.status() == 'DONE' ):
    print(job.status()) 
    sleep(120)
print(job.status()) 
results = job.result() 

This is to retrieve the data from a previous experiment.

In [ ]:
# job = service.job('')  
# results = job.result() 

Now we use the estimated probabilities to estimate the two-qubit Hamiltonians. The prints are the three largest and the lowest eigenvalues of the two-qubit Choi matrices.

In [ ]:
probs = [ prob.data.c.get_counts() for prob in results ]
Hs = hamil_learn.hamiltonian_from_tomo( probs ) 

Passing to the Pauli basis.

In [ ]:
print( 'Theory' )
print( hs[0], hs[0], Js[0])
print('------------------')
print( 'Experiment' )
for j in range(num_qubits-1):
    pauli_coef_tomo = Pauli_Transfer_Matrix( Hs[j], num_qubits=2 )
    # print( pauli_coef_tomo.round(3) )
    print( pauli_coef_tomo[0,-1],pauli_coef_tomo[-1,0],pauli_coef_tomo[1,1])
